In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import time
import random
from bs4 import BeautifulSoup

In [2]:
from selenium.webdriver.support.wait import WebDriverWait
def document_initialised(driver):
    return driver.execute_script("return initialised")

In [24]:
driver = webdriver.Chrome()

In [29]:
all_posts = []
while len(all_posts) == 0:
    try:
        search_query = "jeeadvanced"
        search_url = "https://www.quora.com/search?q="+search_query+"&type=post"
        driver.get(search_url)

        all_posts = driver.find_elements(By.CSS_SELECTOR, ".q-box.qu-borderBottom.qu-px--medium.qu-pt--medium")

        print(len(all_posts))
    except:
        time.sleep(2)
        continue

0
10


In [30]:
scraped_posts = []

In [31]:
while True:
    for post in all_posts:
        # scroll the page till next post is visible and wait for 2 seconds
        driver.execute_script("arguments[0].scrollIntoView();", post)
        time.sleep(2)
        
        videos = post.find_elements(By.CSS_SELECTOR, "div.q-box.qu-mt--small.standalone_featurable")
        for video in videos:
            driver.execute_script("""
            var element = arguments[0];
            element.parentNode.removeChild(element);
            """, video)
            time.sleep(2)
        
        # click on the post to get expanded view
        actions = webdriver.ActionChains(driver)
        # click on top
        actions.move_to_element_with_offset(post, 0, post.size["height"]/2-10)
        actions.click()
        actions.perform()
        
        time.sleep(0.5)
        
        # remove all images and videos from the post
        images = post.find_elements(By.TAG_NAME, "img")
        for image in images:
            driver.execute_script("""
            var element = arguments[0];
            element.parentNode.removeChild(element);
            """, image)
            time.sleep(1)
        
        time.sleep(1)
        
        # scrape the text
        WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR,".q-text.qu-display--block.qu-wordBreak--break-word.qu-textAlign--start")) # this is the p tag containing the text
        
        # get the text from the p tag
        text = driver.find_element(By.CSS_SELECTOR, ".q-text.qu-display--block.qu-wordBreak--break-word.qu-textAlign--start").text
        
        time.sleep(1)
        
        # get the community name
        WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.CSS_SELECTOR, ".q-text.puppeteer_test_tribe_name"))
        
        community_name = driver.find_element(By.CSS_SELECTOR, ".q-text.puppeteer_test_tribe_name").text
        
        time.sleep(1)
        
        # get all the links in the post
        all_links = post.find_elements(By.XPATH, ".//a[@href]")
        
        # get the link to the profile. The href should contain the word profile
        profile_link = ""
        for link in all_links:
            if "profile" in link.get_attribute("href"):
                profile_link = link.get_attribute("href")
                break
        
        time.sleep(1)
        
        # find upvote text and click on it
        try:
            upvote_text = post.find_element(By.XPATH, "//*[text()='View upvotes']")
            # while upvote text is not visible, scroll the page slowly till it is visible 
            # print(upvote_text.text)
            # while not upvote_text.is_displayed():
            #     driver.execute_script("window.scrollBy(0, 100)")
            #     time.sleep(1)
            time.sleep(1)
            upvote_text.click()

            # wait for the upvote popup to load 
            time.sleep(2)

            # get the upvote popup
            upvote_popup = driver.find_element(By.CSS_SELECTOR, "div.q-box.qu-overflowY--auto.qu-display--flex.qu-flexDirection--column.ScrollBox___StyledBox-sc-1t8bc7j-0.eEjJKQ")
            
            # select all links in the popup
            upvoter_links = upvote_popup.find_elements(By.XPATH, ".//a[@href]")

            upvoter_links = [link.get_attribute("href") for link in upvoter_links if "profile" in link.get_attribute("href")]
            
            upvoter_links = list(set(upvoter_links))
            
            time.sleep(3)
            
            # close the popup
            close_button = driver.find_element(By.CLASS_NAME, "q-click-wrapper")
            close_button.click()

            time.sleep(1)
        except:
            upvoter_links = []
            pass
        
        comments_container = post.find_element(By.CLASS_NAME, "comment_and_ad_container")
        comments_container.click()
        
        # now, comments container is expanded. Click on view more comments
        time.sleep(1)
        
        try:
            view_more_comments = comments_container.find_element(By.XPATH, "//*[text()='View more comments']")
            time.sleep(1)
            view_more_comments.click()
        except:
            pass
        
        # now get the profile links of all the commenters
        time.sleep(1)
        
        comment_links = comments_container.find_elements(By.XPATH, ".//a[@href]")
        comment_links = [link.get_attribute("href") for link in comment_links if "profile" in link.get_attribute("href")]
        comment_links = list(set(comment_links))
        
        # remove the profile link of the post author
        comment_links = [link for link in comment_links if link != profile_link]
        
        scraped_posts.append({
            "text": text,
            "community_name": community_name,
            "profile_link": profile_link,
            "upvoter_links": upvoter_links,
            "comment_links": comment_links
        })
        
        # remove post from DOM so that it doesn't interfere with the next post
        
        driver.execute_script("""
        var element = arguments[0];
        element.parentNode.removeChild(element);
        """, post)

        time.sleep(1)
    
    # once posts are finished, get more posts. no need to scroll down as the page is already scrolled down
    try:
        all_posts = driver.find_elements(By.CSS_SELECTOR, ".q-box.qu-borderBottom.qu-px--medium.qu-pt--medium")
        print(len(all_posts))
    except:
        time.sleep(2)
        continue

" Crack JEE Advanced with Maths Dada's Ultimate Challenge! Can you solve it? #
JEEAdvanced
#Mathematics
#ChallengeAccepted
"
JEE Prep with Maths Dada
https://www.quora.com/profile/Maths-Dada
['https://www.quora.com/profile/Study-Account-187', 'https://www.quora.com/profile/Arasada-Navya-Sree', 'https://www.quora.com/profile/Arman-Karn', 'https://www.quora.com/profile/Pyscho-12', 'https://www.quora.com/profile/Whiteshadow-24', 'https://www.quora.com/profile/Sho-680', 'https://www.quora.com/profile/Fahad-Ibrar-4', 'https://www.quora.com/profile/Young-Dumb-Stupid', 'https://www.quora.com/profile/Srashti-20', 'https://www.quora.com/profile/Minato-170', 'https://www.quora.com/profile/Naman-Sharma-1348', 'https://www.quora.com/profile/Aditya-6678', 'https://www.quora.com/profile/Shabnam-Devi-8']
['https://www.quora.com/profile/ARYAN-GUPTA-2856', 'https://www.quora.com/profile/Yuvan-Sujit', 'https://www.quora.com/profile/Shreyas-Srivastava-46', 'https://www.quora.com/profile/Uday-Veer-200', '

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[text()='View upvotes']"}
  (Session info: chrome=116.0.5845.140); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102bde65c chromedriver + 4318812
1   chromedriver                        0x0000000102bd6d00 chromedriver + 4287744
2   chromedriver                        0x00000001028087ec chromedriver + 296940
3   chromedriver                        0x0000000102846048 chromedriver + 548936
4   chromedriver                        0x000000010283b858 chromedriver + 505944
5   chromedriver                        0x000000010287ed28 chromedriver + 781608
6   chromedriver                        0x000000010283a178 chromedriver + 500088
7   chromedriver                        0x000000010283afc0 chromedriver + 503744
8   chromedriver 

In [32]:
print(len(scraped_posts))

6
